In [21]:
from google.transit import gtfs_realtime_pb2
import requests
import pandas as pd

In [22]:
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://bustime.ttc.ca/gtfsrt/vehicles')
feed.ParseFromString(response.content)
current_time = pd.Timestamp.now().time()

In [23]:
def parse_feed_and_filter_by_route(target_route_id):
  print(target_route_id)
  matching_entities = []
  for entity in feed.entity:
      if entity.HasField("vehicle"):
        vehicle = entity.vehicle
        if vehicle.trip.route_id == str(target_route_id):
            print(vehicle.trip)
            matching_entities.append({
                "id": entity.id,
                "trip_id": vehicle.trip.trip_id,
                "route_id": vehicle.trip.route_id,
                "latitude": vehicle.position.latitude,
                "longitude": vehicle.position.longitude,
                "occupancy_status": vehicle.occupancy_status
            })
  return matching_entities

In [24]:
target_route_id = 510
output = 'data_out'
vehicles_on_route = parse_feed_and_filter_by_route(target_route_id)

510
trip_id: "149716020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "150212020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "158615020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "7529020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "104641020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "5759020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "119434020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "27382020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "189837020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "58319020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "61990020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "28309020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "106125020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "183551020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "

In [25]:
# Specify which keys correspond to DataFrame columns
columns = ['vehicle_id', 'trip_id', 'route_id', 'vehicle_latitude', 'vehicle_longitude']

# Mapping from dictionary keys to desired column names
key_to_column = {
    "id": "vehicle_id",
    "trip_id": "trip_id",
    "route_id": "route_id",
    "latitude": "vehicle_latitude",
    "longitude": "vehicle_longitude"
}

# Create the DataFrame, remove duplicate vehicles
df = pd.DataFrame(
    [{new_key: d.get(old_key) for old_key, new_key in key_to_column.items()} for d in vehicles_on_route])
df.drop_duplicates()

# Add current timestampt to df
df['now'] = current_time

# Count unique trip_ids
unique_trip_count = df["trip_id"].nunique()
# print(unique_trip_count)

# Save to csv
df.to_csv(output + '/' + str(target_route_id) + "-rt.csv")